In [30]:
import pandas as pd
import numpy as mp
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score,recall_score,f1_score, roc_auc_score, roc_curve

#load dataset
data = pd.read_csv('E:\\College\\Python\\Churn_Modelling.csv')

#Display the first few rows
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [31]:
# Check for missing values
data.isnull().sum()

# Encode categorical variables
label_encoder = LabelEncoder()
data['Geography'] = label_encoder.fit_transform(data['Geography'])
data['Gender'] = label_encoder.fit_transform(data['Gender'])

# Feature scaling
scaler = StandardScaler()
data[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']] = scaler.fit_transform(data[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']])

# Drop unnecessary columns
data = data.drop(columns=['RowNumber', 'CustomerId', 'Surname'])

# Define feature matrix X and target vector y
X = data.drop(columns=['Exited'])
y = data['Exited']

In [32]:
#Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
# Logistic Regression
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict (X_test)
#Random Forest
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict (X_test)
#Gradient Boosting
gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)
y_pred_gb = gb.predict (X_test)

In [34]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score

# Function to evaluate models
def evaluate_model(y_test, y_pred):
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))
    roc_auc = roc_auc_score(y_test, y_pred)
    print("ROC AUC Score:", roc_auc)
    return roc_auc

# Evaluate Logistic Regression
print("Logistic Regression:")
roc_auc_log_reg = evaluate_model(y_test, y_pred_log_reg)

# Evaluate Random Forest
print("\nRandom Forest:")
roc_auc_rf = evaluate_model(y_test, y_pred_rf)

# Evaluate Gradient Boosting
print("\nGradient Boosting:")
roc_auc_gb = evaluate_model(y_test, y_pred_gb)


Logistic Regression:
Accuracy: 0.815
Confusion Matrix:
 [[1559   48]
 [ 322   71]]
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.97      0.89      1607
           1       0.60      0.18      0.28       393

    accuracy                           0.81      2000
   macro avg       0.71      0.58      0.59      2000
weighted avg       0.78      0.81      0.77      2000

ROC AUC Score: 0.5753961279453282

Random Forest:
Accuracy: 0.8675
Confusion Matrix:
 [[1550   57]
 [ 208  185]]
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.96      0.92      1607
           1       0.76      0.47      0.58       393

    accuracy                           0.87      2000
   macro avg       0.82      0.72      0.75      2000
weighted avg       0.86      0.87      0.85      2000

ROC AUC Score: 0.7176340469732452

Gradient Boosting:
Accuracy: 0.8655
Confusion Matrix:
 [[1547   60]

In [35]:
# Let's assume Gradient Boosting has the highest ROC AUC score
# Hyperparameter tuning for Gradient Boosting
from sklearn.model_selection import GridSearchCV  # Fixed import error

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}

# Instantiate the grid search
grid_search = GridSearchCV(estimator=gb, param_grid=param_grid, cv=3, scoring='roc_auc', verbose=2, n_jobs=-1)  # Fixed syntax errors

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Best parameters
print("Best parameters found: ", grid_search.best_params_)

# Train the optimized model
best_gb = grid_search.best_estimator_  # Fixed assignment operator
best_gb.fit(X_train, y_train)  # Fixed incorrect variable names

# Make predictions with the optimized model
y_pred_best_gb = best_gb.predict(X_test)

# Evaluate the optimized model
print("\nOptimized Gradient Boosting:")
roc_auc_best_gb = evaluate_model(y_test, y_pred_best_gb)  # Fixed function call syntax


Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best parameters found:  {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}

Optimized Gradient Boosting:
Accuracy: 0.8655
Confusion Matrix:
 [[1547   60]
 [ 209  184]]
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.96      0.92      1607
           1       0.75      0.47      0.58       393

    accuracy                           0.87      2000
   macro avg       0.82      0.72      0.75      2000
weighted avg       0.86      0.87      0.85      2000

ROC AUC Score: 0.7154283660385305


In [36]:
# Predicting the Test set results
y_pred_final = best_gb.predict(X_test)  # Fixed syntax error

# Evaluate the final model
print("\nFinal Model Evaluation:")
evaluate_model(y_test, y_pred_final)  # Function call is correct



Final Model Evaluation:
Accuracy: 0.8655
Confusion Matrix:
 [[1547   60]
 [ 209  184]]
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.96      0.92      1607
           1       0.75      0.47      0.58       393

    accuracy                           0.87      2000
   macro avg       0.82      0.72      0.75      2000
weighted avg       0.86      0.87      0.85      2000

ROC AUC Score: 0.7154283660385305


np.float64(0.7154283660385305)